### *Импорты* ###

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential, load_model

### *Загрузка данных* ###

In [2]:
data = pd.read_csv('../ann/../../db/basel(1940-2023).csv')

### *Отделение признаков и таргета в отдельные датафреймы* ###

In [3]:
# Необходимо удалить предсказываемые значения из датафрейма
features = data.drop(['Temperature [2 m elevation corrected]'], axis=1)

In [4]:
target = data['Temperature [2 m elevation corrected]']

### *Разделение на обучающий и тестовый наборы* ###

In [5]:
# Обучающая часть
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=True)

### *Стандартизация данных* ###

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### *Инициализация модели* ###

In [7]:
model = Sequential([
    layers.Dense(128, activation='tanh', input_shape=(X_train_scaled.shape[1], ), kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.05),

    layers.Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.05),

    layers.Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.05),

    layers.Dense(1, kernel_regularizer=regularizers.l2(0.01)),
])

### *Планировщик скорости обучения* ###

In [8]:
# Слишком малое количество прогревочных эпох может привести к тому, что скорость обучения будет расти слишком быстро, 
# а слишком большое их количество может замедлить сходимость модели
warmup_epochs = 5

# Если начальная скорость обучения слишком высока, это может привести к расхождению модели, а если слишком мала - модель 
# будет сходиться очень медленно
initial_lr = 1e-4

# Шаги распада определяют, как быстро уменьшается скорость обучения
decay_steps = 10

def lr_schedule(epoch):
    if epoch < warmup_epochs:
        lr = (epoch + 1) / warmup_epochs * initial_lr
        
    else:
        lr = initial_lr * (0.5 ** ((epoch - warmup_epochs) / decay_steps))
    
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [9]:
# Определяет насколько сильно должна быть снижена скорость обучения, 'patience' - количество эпох без улучшений, 
# после которых скорость обучения будет снижена, а 'min_lr' - нижняя граница скорости обучения
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=25, min_lr=1e-6)

In [10]:
# Определяет количество эпох без улучшений, после которых обучение будет остановлено
early_stopping = EarlyStopping(monitor='loss', patience=25, restore_best_weights=True)

### *Применение оптимизатора и компиляция модели* ###

In [11]:
# Stochastic Gradient Descent - оптимизационный алгоритм, используемый для обучения моделей машинного обучения. 
# Основная идея SGD заключается в том, чтобы минимизировать функцию потерь, обновляя параметры модели в направлении, 
# противоположном градиенту функции потерь
sgd_optimizer = SGD(learning_rate=0.01)
model.compile(optimizer=sgd_optimizer, loss='mean_absolute_error')

### *Обучение модели* ###

In [12]:
model.fit(X_train_scaled, y_train, epochs=15, batch_size=16, callbacks=[lr_scheduler, reduce_lr, early_stopping])

Epoch 1/15
36774/36774 [==============================] - 51s 1ms/step - loss: 11.6056 - lr: 2.0000e-05
Epoch 2/15
36774/36774 [==============================] - 47s 1ms/step - loss: 6.2562 - lr: 4.0000e-05
Epoch 3/15
36774/36774 [==============================] - 51s 1ms/step - loss: 4.2548 - lr: 6.0000e-05
Epoch 4/15
36774/36774 [==============================] - 48s 1ms/step - loss: 3.9492 - lr: 8.0000e-05
Epoch 5/15
36774/36774 [==============================] - 51s 1ms/step - loss: 3.6665 - lr: 1.0000e-04
Epoch 6/15
36774/36774 [==============================] - 48s 1ms/step - loss: 3.4138 - lr: 1.0000e-04
Epoch 7/15
36774/36774 [==============================] - 45s 1ms/step - loss: 3.2111 - lr: 9.3303e-05
Epoch 8/15
36774/36774 [==============================] - 48s 1ms/step - loss: 3.0416 - lr: 8.7055e-05
Epoch 9/15
36774/36774 [==============================] - 46s 1ms/step - loss: 2.9090 - lr: 8.1225e-05
Epoch 10/15
36774/36774 [==============================] - 48s 1ms/step 

### *Результативность модели* ###

In [13]:
prediction = model.predict(X_test_scaled)

print('Mean Squared Error: {:.4f}'.format(mean_squared_error(y_test, prediction)))
print('Mean Absolute Error: {:.4f}'.format(mean_absolute_error(y_test, prediction)))
print('R-squared: {:.4f}'.format(r2_score(y_test, prediction)))

4597/4597 [==============================] - 6s 1ms/step
Mean Squared Error: 1.2553
Mean Absolute Error: 0.7893
R-squared: 0.9811


### *Сохранение модели* ###

In [14]:
model.save('best_model_full_year.keras')

### *Загрузка модели* ###

In [15]:
# loaded_model = load_model('best_model.keras')